# CHM210 Chemistry of Environmental Change


This is our introductory Jupyter notebook! Notebooks should start with a written introduction of the context and content that will be covered in the assignment. We can include inspiring images here like this GEOS-Chem simulation of global ozone from <a href="http://fizz.phys.dal.ca/~atmos/martin/?page_id=100">Randall Martin's group</a>:

<img src="http://fizz.phys.dal.ca/~atmos/animation/o3_movie.gif">


References can be <a href="http://www.chem.utoronto.ca/">linked</a> <a href="http://www.chem.utoronto.ca/grad/">here</a> to <a href="http://www.chem.utoronto.ca/undergrad/">resources</a> that will help students with the assignment. 

To run code, press play/run or hold Ctrl-Enter on the simple example below:

In [ ]:
 print("this is an example of a simple print statement")

It's straightforward to do simple arithmetic,  

In [ ]:
5+10

And symbolic mathematics is easy to represent through LaTeX, ie. $e^{i\pi} + 1 = 0$. 

Chemical reactions are straightforward to write in the same way: $2X_{(g)} \rightarrow 3Y_{(g)} + 4Z_{(g)}$

Simple markdown structures like tables are also easy to build:

| Reaction | 2X   | 3y   | 4z   |
|------|------|------|------|
|   Initial amounts  | table| table| table|
|   Change in amount  | table| table| table|
|   Equilibrium amount  | table| table| table|

We can do more advanced things where we call Python to help us.

In [97]:
import sys
!{sys.executable} -m pip install scipy
%matplotlib inline
from ipywidgets import interactive, fixed

Here are the Lorenz Equations
\begin{align}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{align}

We can call in demos that we've built and saved outside the notebook like so, (run the below code by pressing the play/run button or by pressing Ctrl-Enter while in the box):

In [ ]:
from lorenz import solve_lorenz
w=interactive(solve_lorenz,sigma=(0.0,50.0),rho=(0.0,50.0))
w

We can easily do simple calculations. The below code will calculate the average positions in  𝑥, 𝑦, and 𝑧 directions for the above figure. If you change any of the input settings, you'll have to re-run the below code to see the results change.

In [ ]:
t, x_t = w.result
xyz_avg = x_t.mean(axis=1)
print(xyz_avg)

We can make all manner of standard plots, like histograms (below is a histogram of average positions in the x-direction)

In [ ]:
from matplotlib import pyplot as plt
plt.hist(xyz_avg[:,0])
plt.title('Average $x(t)$');

It's also straightforward to load in outside data

In [96]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame, Panel
pd.set_option('display.max_rows',15) # this limit maximum numbers of rows

Let's load some data from NOAA on the Arctic Oscillation (AO) and North Atlantic Oscillation (NAO) data sets (this data was provided here: https://nbviewer.jupyter.org/github/koldunovn/earthpy.org/blob/master/content/earthpy_pandas_basics.ipynb).

In [ ]:
#!curl http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii >> 'monthly.ao.index.b50.current.ascii'
!wget http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii
ao = np.loadtxt('monthly.ao.index.b50.current.ascii')

Let's look at some time series plots now (code will be heavily commented for student use, this is just a quick demo of capabities)

In [ ]:
dates = pd.date_range('1950-01', periods=ao.shape[0], freq='M')
AO = Series(ao[:,2], index=dates)
AO.plot()

We can look at subplots easily

In [ ]:
AO['1980-05':'1981-03'].plot()

And we can compare different time series (we'll load up some more data first)

In [ ]:
#!curl http://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/norm.nao.monthly.b5001.current.ascii >> 'norm.nao.monthly.b5001.current.ascii'
!wget http://www.cpc.ncep.noaa.gov/products/precip/CWlink/pna/norm.nao.monthly.b5001.current.ascii
nao = np.loadtxt('norm.nao.monthly.b5001.current.ascii')
dates_nao = pd.date_range('1950-01', periods=nao.shape[0], freq='M')
NAO = Series(nao[:,2], index=dates_nao)
aonao = DataFrame({'AO' : AO, 'NAO' : NAO})

And now plot them both side by side

In [ ]:
aonao.plot(subplots=True)

We can look at more complex data visualizations too (example from http://earthpy.org/05_Graphs_and_maps_Matplotlib_and_Basemap.html)

In [ ]:
from scipy.io import netcdf
!wget http://schubert.atmos.colostate.edu/~cslocum/code/air.sig995.2012.nc
f =netcdf.netcdf_file('air.sig995.2012.nc')

Now we can plot global air temperature from the NMC reanalysis

In [ ]:
air = f.variables['air']
lat = f.variables['lat'][:]
lon = f.variables['lon'][:]
air_c = ((air[:] * air.scale_factor) + air.add_offset) - 273.15
imshow(air_c[0,:,:])
colorbar()

We can also include complex code to look at fun things like gas-phase kinetics. Below is the code created by Elijah G. Schnitzler for CHM210 in 2018 as part of the CTFP program. (I'll need to add a "codefolding" command in here so the entire piece of code doesn't display at once).

In [ ]:
# Step 1: Select model condition:
#   1. NO2 photolysis
#   2. NOx cycle + O3 cycle
#   3. NOx cycle + O3 cycle + VOC reactions

Condition = 3 #select 1-3

# Step 2: Adjust initial mixing ratios (in ppb).

NO2 = 1
NO = 10
VOC = 20

# Note that the VOC mixing ratio is divided equally between alkanes and aldehydes.
# Please do not adjust parameters below this line.

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from math import exp

cNO2i = NO2*0.001
cNOi = NO*0.001
cRHi = VOC/2*0.001
cRCHOi = VOC/2*0.001

T = 298

if Condition < 3:
  Time = 60
  Step = 0.1
else:
  Time = 600
  Step = 1

if Condition == 1:
  def Rxn(Z,t):
      # All rate constants and rates are expressed in units of min and ppm.
      k_1 = 0.5 # no2 + hv
      J_1 = k_1 * Z[0] # no2
      dNO2dt = - J_1
      return[dNO2dt]
  
  t = np.arange(0,Time,Step)
  
  Z0 = [cNO2i]
  Conc = odeint(Rxn,Z0,t)
  
  cNO2 = Conc[:,0]*1000
  
  if cNO2[599] > 1:
    cNO2f = float("{0:.1f}".format(cNO2[599]))
  else:
    cNO2f = float("{0:.1f}".format(0))

  plt.plot(t,cNO2)

  plt.title('Mixing ratios of constituents of photochemical smog')
  plt.xlabel('Reaction time / min')
  plt.ylabel('Mixing ratio / ppb')
  plt.legend(('NO2',))
  plt.annotate(cNO2f,(55,cNO2[599]),color='C0',weight='bold')
  plt.show()

if Condition == 2:
  def Rxn(Z,t):
      # All rate constants and rates are expressed in units of min and ppm.
      k_1 = 0.5 # no2 + hv
      k_2 = 60*2.46E13*2.46E13*6.0E-34*(T/300)**(-2.3) # o + o2 (+ m)
      k_3 = 60*2.46E13*2.2E-12*exp(-1430/T) # no + o3
      J_1 = k_1 * Z[0] # no2
      J_2 = k_2 * Z[1] * Z[2] # o * o2
      J_3 = k_3 * Z[3] * Z[4] # no * o3
      dNO2dt = - J_1 + J_3
      dOdt = J_1 - J_2
      dO2dt = - J_2 + J_3
      dNOdt = J_1 - J_3
      dO3dt = J_2 - J_3
      return[dNO2dt,dOdt,dO2dt,dNOdt,dO3dt]
  
  t = np.arange(0,Time,Step)
  
  Z0 = [cNO2i,0,0.21E6,cNOi,0]
  Conc = odeint(Rxn,Z0,t)
  
  cNO2 = Conc[:,0]*1000
  cO = Conc[:,1]*1000
  cO2 = Conc[:,2]*1000
  cNO = Conc[:,3]*1000
  cO3 = Conc[:,4]*1000
  
  cNO2f = float("{0:.1f}".format(cNO2[599]))
  cNOf = float("{0:.1f}".format(cNO[599]))
  cO3f = float("{0:.1f}".format(cO3[599]))
  
  plt.plot(t,cNO2)
  #plt.plot(t,cO)
  #plt.plot(t,cO2)
  plt.plot(t,cNO)
  plt.plot(t,cO3)
  
  plt.title('Mixing ratios of constituents of photochemical smog')
  plt.xlabel('Reaction time / min')
  plt.ylabel('Mixing ratio / ppb')
  plt.legend(('NO2','NO','O3'))
  plt.annotate(cNO2f,(55,cNO2[599]),color='C0',weight='bold')
  plt.annotate(cNOf,(55,cNO[599]),color='C1',weight='bold')
  plt.annotate(cO3f,(55,cO3[599]),color='C2',weight='bold')
  plt.show()

if Condition == 3:
  def Rxn(Z,t):
      # All rate constants and rates are expressed in units of min and ppm.
      k_1 = 0.5 # no2 + hv
      k_2 = 60*2.46E13*2.46E13*6.0E-34*(T/300)**(-2.3) # o + o2 (+ m)
      k_3 = 60*2.46E13*2.2E-12*exp(-1430/T) # no + o3
      k_4 = 60*2.46E13*1.68E-11*exp(-559/T) # rh + oh
      k_5a = 60*2.46E13*6.9E-12*exp(250/T) # rcho + oh
      k_5b = 1.91E-4 # rcho + hv
      k_6 = 60*2.46E13*3.7E-12*exp(240/T) # ho2 + no
      k_7 = 60*2.46E13*4.2E-12*exp(180/T) # ro2 + no
      k_8 = 60*2.46E13*4.2E-12*exp(180/T) # rc(o)o2 + no
      k_9 = 60*2.46E13*1.1E-11 # oh + no2
      k_10 = 60*2.46E13*4.7E-12 # rc(o)o2 + no2
      k_11 = 60*1.95E16*exp(-13543/T) # rc(o)o2no2 decomposition
      J_1 = k_1 * Z[0] # no2
      J_2 = k_2 * Z[1] * Z[2] # o * o2
      J_3 = k_3 * Z[3] * Z[4] # no * o3
      J_4 = k_4 * Z[5] * Z[6] # rh * oh
      J_5a = k_5a * Z[7] * Z[6] # rcho * oh
      J_5b = k_5b * Z[7] # rcho
      J_6 = k_6 * Z[8] * Z[3] # ho2 * no
      J_7 = k_7 * Z[9] * Z[3] # ro2 * no
      J_8 = k_8 * Z[10] * Z[3] # rc(o)o2 * no
      J_9 = k_9 * Z[6] * Z[0] # oh * no2
      J_10 = k_10 * Z[10] * Z[0] # rc(o)o2 * no2
      J_11 = k_11 * Z[11] # rc(o)o2no2
      dNO2dt = - J_1 + J_3 + J_6 + J_7 + J_8 - J_9 - J_10 + J_11
      dOdt = J_1 - J_2
      dO2dt = - J_2 + J_3
      dNOdt = J_1 - J_3 - J_6 - J_7 - J_8
      dO3dt = J_2 - J_3
      dRHdt = - J_4
      dOHdt = - J_4 - J_5a + J_6 - J_9
      dRCHOdt = - J_5a - J_5b + J_7
      dHO2dt = J_5b - J_6 + J_7
      dRO2dt = J_4 + J_5b - J_7 + J_8
      dRCOO2dt = J_5a - J_8 - J_10 + J_11
      dRCOO2NO2dt = J_10 - J_11
      dHNO3dt = J_9
      return[dNO2dt,dOdt,dO2dt,dNOdt,dO3dt,dRHdt,dOHdt,dRCHOdt,dHO2dt,dRO2dt,dRCOO2dt,dRCOO2NO2dt,dHNO3dt]
  
  t = np.arange(0,Time,Step)
  
  Z0 = [cNO2i,0,0.21E6,cNOi,0,cRHi,0,cRCHOi,0,0,0,0,0]
  Conc = odeint(Rxn,Z0,t)
  
  cNO2 = Conc[:,0]*1000
  cO = Conc[:,1]*1000
  cO2 = Conc[:,2]*1000
  cNO = Conc[:,3]*1000
  cO3 = Conc[:,4]*1000
  cRH = Conc[:,5]*1000
  cOH = Conc[:,6]*1000
  cRCHO = Conc[:,7]*1000
  cHO2 = Conc[:,8]*1000
  cRO2 = Conc[:,9]*1000
  cRCOO2 = Conc[:,10]*1000
  cRCOO2NO2 = Conc[:,11]*1000
  cHNO3 = Conc[:,12]*1000
  
  cNO2f = float("{0:.1f}".format(cNO2[599]))
  cNOf = float("{0:.1f}".format(cNO[599]))
  cO3f = float("{0:.1f}".format(cO3[599]))
  cRHf = float("{0:.1f}".format(cRH[599]))
  cRCHOf = float("{0:.1f}".format(cRCHO[599]))
  
  plt.plot(t,cNO2)
  #plt.plot(t,cO)
  #plt.plot(t,cO2)
  plt.plot(t,cNO)
  plt.plot(t,cO3)
  plt.plot(t,cRH)
  plt.plot(t,cRCHO)
  #plt.plot(t,cOH)
  #plt.plot(t,cHO2)
  #plt.plot(t,cRCOO2)
  #plt.plot(t,cRCOO2NO2)
  #plt.plot(t,cHNO3)
  
  plt.title('Mixing ratios of constituents of photochemical smog')
  plt.xlabel('Reaction time / min')
  plt.ylabel('Mixing ratio / ppb')
  plt.legend(('NO2','NO','O3','RH','RCHO'))
  plt.annotate(cNO2f,(550,cNO2[599]),color='C0',weight='bold')
  plt.annotate(cNOf,(550,cNO[599]),color='C1',weight='bold')
  plt.annotate(cO3f,(550,cO3[599]),color='C2',weight='bold')
  plt.annotate(cRHf,(550,cRH[599]),color='C3',weight='bold')
  plt.annotate(cRCHOf,(550,cRCHO[599]),color='C4',weight='bold')
  plt.show()
